# 1.2.1 Phonon Calculation: Displacing the Atoms along Normal Modes.

Once a Cartesian Phonon Calculation is performed and Normal modes are obtained, phonons can be recomputed by displacing the atoms along those normal modes. A phonon calculated along normal mode displacements have less numerical noise, particularly for low frequency modes and these frequencies give numerically more accurate free energies, see for example the following two papers:
(1) https://doi.org/10.1021/ct500291x &
(2) https://doi.org/10.1021/jacs.6b08646.

To calculate electron phonon coupling energies (EPCE), which are defined as the second derivative of the band energies (Kohn-Sham eigenvalues) with respect to a particular phonon mode, see Eq. S2 of the following paper: https://doi.org/10.1103/PhysRevMaterials.5.L070801. The sum of EPCEs over all phonon modes is the zero-point renormalization (ZPR) of that band. To compute EPCEs or ZPR using a frozen phonon method, it is necessary to recompute phonons by displacing the atoms along normal modes. In this exercise we will learn that. 

The first step is to prepare the displaced coordinates. For that purpose again we import the following coordinate utilities tools, electron-phonon classes and pyepfd input/output tools from pyepfd.

In [1]:
from pyepfd.coord_util import *
from pyepfd.elph_classes import *
from pyepfd.pyepfd_io import *

          ███████████                            
         ░░███░░░░░███                           
          ░███    ░███ █████ ████                
          ░██████████ ░░███ ░███                 
          ░███░░░░░░   ░███ ░███                 
          ░███         ░███ ░███                 
          █████        ░░███████                 
         ░░░░░          ░░░░░███                 
                        ███ ░███                 
                       ░░██████                  
                        ░░░░░░                   
 ██████████ ███████████  ███████████ ██████████  
░░███░░░░░█░░███░░░░░███░░███░░░░░░█░░███░░░░███ 
 ░███  █ ░  ░███    ░███ ░███   █ ░  ░███   ░░███
 ░██████    ░██████████  ░███████    ░███    ░███
 ░███░░█    ░███░░░░░░   ░███░░░█    ░███    ░███
 ░███ ░   █ ░███         ░███  ░     ░███    ███ 
 ██████████ █████        █████       ██████████  
░░░░░░░░░░ ░░░░░        ░░░░░       ░░░░░░░░░░   
PyEPFD version     :  1.0
Author             : Arp

Next, we will read all information from the checkpoint/restart XML file created in the previous step, i.e., Cartesian phonon calculation. We would instantiate an object named enmfd_inp of class read_pyepfd_info that belongs to coord_util object. The class read_pyepfd_info has several objects such as acoustic sum rule (asr), refined dynamical matrix (ref_dynmatrix), optimized coordinates(coord), atoms and cell.  

In [2]:
enmfd_inp = read_pyepfd_info(\
            file_path='../1_cartesian_phonon/fdphonon.xml')

Time spent on read_pyepfd_info class: 0.0005269050598144531 s.


Now we would save these informations into different variables.

In [3]:
asr = enmfd_inp.asr
inp_dynmat = enmfd_inp.ref_dynmatrix
opt_coord = enmfd_inp.coord
atoms = enmfd_inp.atoms
cell = enmfd_inp.cell

Like Cartesian phonon calculation, we again instantiate an object named fdmoves of ionic_mover class. Here we would use mode = 'ENMFD', i.e. energy-scaled normal mode finite displacement. Here we also need to provide the dynamical matrix (key: dynmat) and target energy displacement (deltae). Our target energy displacement is 0.001 Ha.  

In [4]:
fdmoves = ionic_mover( atoms = atoms,\
                      opt_coord = opt_coord,\
                      mode = 'ENMFD', \
                      dynmat = inp_dynmat,\
                      deltax = 0.005,\
                      deltae = 0.001 )

#Mode =      1 Disp-step(au) =    81.7641 Disp-step(Freq-scaled) =        nan
#Config    Disp(au)   Disp(Freq-scaled)
 2       81.7641           nan
 3      -81.7641           nan
#Mode =      2 Disp-step(au) =    84.5714 Disp-step(Freq-scaled) =        nan
#Config    Disp(au)   Disp(Freq-scaled)
 4       84.5714           nan
 5      -84.5714           nan
#Mode =      3 Disp-step(au) =    85.3889 Disp-step(Freq-scaled) =        nan
#Config    Disp(au)   Disp(Freq-scaled)
 6       85.3889           nan
 7      -85.3889           nan
#Mode =      4 Disp-step(au) =    81.7642 Disp-step(Freq-scaled) =     0.0162
#Config    Disp(au)   Disp(Freq-scaled)
 8       81.7642        0.0162
 9      -81.7642       -0.0162
#Mode =      5 Disp-step(au) =    82.5022 Disp-step(Freq-scaled) =     0.0239
#Config    Disp(au)   Disp(Freq-scaled)
 10       82.5022        0.0239
 11      -82.5022       -0.0239
#Mode =      6 Disp-step(au) =    15.3008 Disp-step(Freq-scaled) =     0.8272
#Config    Disp(au) 

/home/arpan/.local/lib/python3.8/site-packages/pyepfd/coord_util.py:676: RuntimeWarning: invalid value encountered in sqrt
  freq_scaling = np.sqrt(nmfd.omega[imode])


First 5 modes are rigid body translation and rotation mode and in frequency scaled coordinate displacement are imaginary. This is okay. 

Now all the displaced coordinates are stored within fdmoves object. Next step is to write them into an xyz file. To do that we instantiate an object named out_xyz of class xyz with input/output mode write(w). 

In [5]:
out_xyz = xyz(file_path = 'enmfd_phonon.xyz',\
              io = 'w', atoms = atoms)

Time spent on xyz class: 0.0002453327178955078 s.


Next we write the enmfd_phonon.xyz file step by step applying a loop.

In [6]:
ndisp = fdmoves.disp_coord.shape[1]
for i in range(ndisp):
    out_xyz.write(cell = cell,coord = fdmoves.disp_coord[:,i])
del out_xyz

This generated the coordinates by displacing the atoms along normal modes and wrote it into enmfd_phonon.xyz file.
Let us have a look at this file.

In [7]:
%%bash
cat enmfd_phonon.xyz

3
# CELL(abcABC):   10.58354    10.58354    10.58354    90.00000    90.00000    90.00000  PyEPFD-Step: 0 positions{angstrom} cell{angstrom}
O                  0                0        1.1571328
O                  0                0       -1.1571313
C                  0                0   -1.5875317e-06
3
# CELL(abcABC):   10.58354    10.58354    10.58354    90.00000    90.00000    90.00000  PyEPFD-Step: 1 positions{angstrom} cell{angstrom}
O     -2.1644539e-08   -1.3546308e-09        1.3098932
O     -1.0825663e-08   -8.5003505e-09       -1.0043709
C      -1.623509e-08    -4.927498e-09       0.15275873
3
# CELL(abcABC):   10.58354    10.58354    10.58354    90.00000    90.00000    90.00000  PyEPFD-Step: 2 positions{angstrom} cell{angstrom}
O      2.1644539e-08    1.3546308e-09        1.0043725
O      1.0825663e-08    8.5003505e-09       -1.3098916
C       1.623509e-08     4.927498e-09       -0.1527619
3
# CELL(abcABC):   10.58354    10.58354    10.58354    90.00000    90.00000    90.00

The next step is to convert them into qbox input. This can be done using xyz2qb available in util/qbox_utils folder. This is shown in example 1.2.2